In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('C:/Users/Iasonas/PycharmProjects/EntityMatching_Thesis/DataFusion/tutorials')

import fuse
fuse_env = fuse.Fuse(verbose=True)
fuse_session = fuse_env.get_session('test')

MATCHER='DEDUPE'
# MATCHER='AHC'

C:\Python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Iasonas\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
C:\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using versi

In [2]:
if MATCHER=='DEDUPE':
    from matching.dedupe import DedupeMatcher
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                                  'fusion_test', matching_strategy=DedupeMatcher)

if MATCHER=='AHC':
    from matching.collective_matching_ahc import AgglomerativeHierarchicalClustering
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                               'fusion_test', matching_strategy=AgglomerativeHierarchicalClustering)


INFO:dedupe.api:Predicate set:
INFO:dedupe.api:(LevenshteinCanopyPredicate: (2, company), TfidfNGramCanopyPredicate: (0.4, year), SimplePredicate: (commonSixGram, company))
INFO:dedupe.api:(SimplePredicate: (firstIntegerPredicate, year), TfidfTextCanopyPredicate: (0.6, title))
INFO:dedupe.api:Predicate set:
INFO:dedupe.api:Predicate set:
INFO:dedupe.api:Predicate set:
INFO:dedupe.api:(TfidfTextCanopyPredicate: (0.6, year), LevenshteinCanopyPredicate: (3, university), LevenshteinCanopyPredicate: (4, university))
INFO:dedupe.api:(TfidfNGramCanopyPredicate: (0.2, university), SimplePredicate: (sameFiveCharStartPredicate, degree))
INFO:dedupe.api:(SimplePredicate: (commonFourGram, year), SimplePredicate: (sameThreeCharStartPredicate, degree), LevenshteinCanopyPredicate: (4, university))
INFO:dedupe.api:(SimplePredicate: (nearIntegersPredicate, year), SimplePredicate: (suffixArray, university))


In [3]:
fuse_session.train_matchers('../../datasets/current/matched.json')

In [5]:
# x=dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','doctoral_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','Bsc')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Msc','Bsc')
# x=dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('manager','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('coder','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('doctor','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('clinician','doctor')

In [6]:
fuse_session.match_observations()

In [7]:
tr_clusters = fuse_session.find_true_clusters()

INFO:global:Running truth discovery for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Working Experience': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}, 'Location': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}, 'Name': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}, 'Education': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}}
INFO:global:Iteration = 1
INFO:global:Source weights = {'Working Experience': {'pipl.com': 4.595119850134589, 'pro-profiles.com': 4.595119850134589, 'internsteps.com': 4.595119850134589, 'find.jobs': 4.595119850134589}, 'Location': {'pipl.com': 4.595119850134589, 'pro-profiles.com': 4.595119850134589, 'internsteps.com': 4.595119850134589, 'find.jobs': 4.595119850134589}, 'Name': {'pipl.com': 4.595119850134589, 'pro-profiles.com': 4.595119850134589, 'internsteps.com': 4.595119850134589,

In [8]:
true_facts = fuse_session.find_true_facts(persist=True, no_weights=False)
for f in true_facts:
    print(f.canonicalTuple)

INFO:gensim.models.utils_any2vec:loading projection weights from C:/Users/Iasonas/Downloads/GoogleNews-vectors-negative300.bin
INFO:gensim.models.utils_any2vec:loaded (3000000, 300) matrix from C:/Users/Iasonas/Downloads/GoogleNews-vectors-negative300.bin
INFO:global:Running fusion for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Working Experience': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}, 'Location': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}, 'Name': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}, 'Education': {'pipl.com': 1.0, 'pro-profiles.com': 1.0, 'internsteps.com': 1.0, 'find.jobs': 1.0}}
INFO:global:Iteration = 1
INFO:global:Source weights = {'Working Experience': {'pipl.com': 4.595119850134589, 'pro-profiles.com': 5.411646051855058, 'internsteps.com': 4.595119850134589, 'find.jobs': 4.595119850134589}, 'Location': {'pipl.com'

{'name': 'Andrew Knapp'}
{'name': 'Christopher Logan'}
{'name': 'S. Juan'}
{'name': 'Tyler Anderson'}
{'name': 'MMauthis'}
{'location': 'Brownton'}
{'location': 'CHRISTINABERG'}
{'name': 'Richard Jordan'}
{'years': '2003-2020', 'title': 'doctor', 'company': 'Olson, Rogers and Brown'}
{'location': 'Elizabethfurt P.'}
{'years': '1993-1996', 'title': 'manager', 'company': 'Martinez-Duran'}
{'name': 'Jaton Smith'}
{'years': '2018-2020', 'title': 'developer', 'company': 'Powers and Sons'}
{'name': 'Anthony Diaaz'}
{'university': 'MIT', 'degree': 'Phd', 'year': '1997'}
{'location': 'GARCIAVILLE'}
{'location': 'M. North'}
{'years': '1996-2008', 'title': 'developer', 'company': 'Roberts, Wilson and Middleton'}
{'location': 'Lake Michael'}
{'name': 'Kimerly EDelacruz'}
{'name': 'DAVE WEST'}
{'name': 'Michael Edxardds'}
{'location': 'Russohaven'}
{'years': '2004-2019', 'title': 'developer', 'company': 'Garrison-Morton'}
{'university': 'UCSC', 'degree': 'Phd', 'year': '2010'}
{'location': 'West A

In [9]:
eval = fuse_session.evaluate(inferred_facts=true_facts,grd_path='../../datasets/current/eval.json')
print(eval)

INFO:gensim.models.utils_any2vec:loading projection weights from C:/Users/Iasonas/Downloads/GoogleNews-vectors-negative300.bin
INFO:gensim.models.utils_any2vec:loaded (3000000, 300) matrix from C:/Users/Iasonas/Downloads/GoogleNews-vectors-negative300.bin


Matched = 194.00, Unmatched true = 6.00, Unmatched inferred = 14.00, Total = 200.00
recall= 0.97
Precision = 0.93
Accuracy = 0.91
f1score = 0.95
{'recall': 0.97, 'precision': 0.9326923076923077, 'f1score': 0.9509803921568627}
